# Model subclassing  
Model 클래스는 fit(), evaluate(), predict() 메서드를 가지고 있으며,  
모델을 디스크에 파일로 저장할 수 있다.  
## Model class를 상속하여 사용자 정의 모델을 만드는 방법

- `__init__()` 메서드에서 모델이 사용할 층을 정의함  
- `call()` 메서드에서 만든 층을 사용하여 모델의 정방향 패스를 정의  
- 서브클래스의 객체를 만들고 데이터와 함께 호출하여 가중치를 만듬

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

이하의 예제는 functional api에서 구현하였던 티켓 관리 예제를 다시 구현한 것이다.

In [9]:
class CustomerTicketModel(keras.Model):
    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(
        num_departments, activation="softmax")
    def call(self, inputs):
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]
        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

모델을 정의했으므로, 이 클래스의 객체를 만들 수 있다. Layer 클래스와 마찬가지로 어떤 데이터로 처음 호출할 때 가중치를 만든다.

In [10]:
import numpy as np
num_samples = 1280
vocabulary_size = 10000
num_tags = 100
num_departments= 4

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [11]:
model = CustomerTicketModel(num_departments=4)
priority, department = model(
    {"title": title_data,
    "text_body": text_body_data,
    "tags": tags_data}
)

모델을 컴파일하고 훈련할 수 있다.

In [13]:
model.compile(optimizer="rmsprop",
             loss=["mean_squared_error", "categorical_crossentropy"],
             metrics=[["mean_absolute_error"], ["accuracy"]])
model.fit({"title": title_data,
         "text_body": text_body_data,
         "tags": tags_data},
         [priority_data, department_data],
         epochs=1)
model.evaluate({"title": title_data,
               "text_body": text_body_data,
                "tags": tags_data},
                [priority_data, department_data]
               )

40/40 [==============================] - 1s 9ms/step - loss: 35.8348 - output_1_loss: 0.3264 - output_2_loss: 35.5084 - output_1_mean_absolute_error: 0.4956 - output_2_accuracy: 0.0555


[35.83479309082031,
 0.3264123797416687,
 35.50837326049805,
 0.49560290575027466,
 0.05546874925494194]

In [16]:
priority_preds, department_preds = model.predict({"title": title_data,
                                                 "text_body": text_body_data,
                                                 "tags": tags_data})
print(priority_preds)
print(department_preds)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
[[7.2048595e-10 2.6752944e-18 8.8690458e-06 9.9999118e-01]
 [6.3865646e-10 2.4473887e-17 2.3659237e-05 9.9997640e-01]
 [1.7777876e-09 1.0094958e-18 7.9579349e-06 9.9999201e-01]
 ...
 [7.5876078e-10 2.2226465e-18 6.0110115e-06 9.9999404e-01]
 [9.2045033e-10 4.4560174e-18 5.7123407e-06 9.9999428e-01]
 [7.7897500e-10 1.1169499e-17 1.9717500e-05 9.9998033e-01]]


모델 서브클래싱으로는 모델을 만들 때 dag(directed acyclic graph)로 표현할 수 없는 모델을 만들 수 있다.  
예를 들어 call() 메서드가 for 루프 안에서 층을 사용하거나 재귀적으로 호출하는 등의 모델도 가능하다.

## 서브클래싱된 모델의 단점
모델 오류 가능성에 따라 디버깅 작업이 많아진다.  
함수형 모델은 명시적 데이터 구조인 층의 그래프이므로, 출력, 조사, 수정이 가능하다.  
서브클래싱 모델은 한 덩어리의 바이트코드(bytecode)로서, 원시 코드가 담긴 call() 메서드를 가진 파이썬 클래스이다.  
이로 인한 유연성이 장점이지만, 새로운 제약이 발생한다.  
예컨대 층이 서로 연결되는 방식이 call()메서드 안에 감추어진다. 이 정보를 활용하기 어렵다.  
summary() 메서드가 층의 연결 구조를 출력할 수 없다. plot_model()함수로 모델의 구조를 그래프로 그릴 수 없다.  

비슷하게 서브클래싱 모델은 그래프가 없기 떄문에 특성 추출을 위해 층 그래프의 노드를 참조할 수 없다.  
이 모델의 객체를 생성하고 나면 정방향 패스는 call()메서드 소스를 직접 확인하지 않고서는 모델 구조를 알 수 없다.
